# Taller básico sobre MONGOdb
---
1. Instalar librerías
2. Descargar e instalar un servidor propio de MongoDb
3. Crear una BD
4. Crear colecciones
5. Descargar .zip (csv) cargarlos en una colección


#1.Instalar librerías

In [1]:
!apt update
!apt install -y python3-pip
!pip3 install pymongo        #libreria especial

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://cli.github.com/packages stable InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:8 https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/6.0 InRelease [4,009 B]
Hit:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Err:8 https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/6.0 InRelease
  The following signatures couldn't be verified because the public key is 

#2.Descargar e instalar un servidor propio de MongoDb

In [2]:
!sudo apt-get install -y gnupg curl
!curl -fsSL https://pgp.mongodb.com/server-6.0.asc | sudo gpg -o /usr/share/keyrings/mongodb-server-6.0.gpg --dearmor
!echo "deb [ arch=amd64,arm64 signed-by=/usr/share/keyrings/mongodb-server-6.0.gpg ] https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/6.0 multiverse" | sudo tee /etc/apt/sources.list.d/mongodb-org-6.0.list

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
curl is already the newest version (7.81.0-1ubuntu1.20).
gnupg is already the newest version (2.2.27-3ubuntu2.4).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
deb [ arch=amd64,arm64 signed-by=/usr/share/keyrings/mongodb-server-6.0.gpg ] https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/6.0 multiverse


In [3]:
!sudo apt-get update
!sudo apt-get install -y mongodb-org

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Get:3 https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/6.0 InRelease [4,009 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/6.0/multiverse amd64 Packages [108 kB]
Get:14 https://repo.mongodb.org/apt/ubuntu jammy

##2.1.Instalar mongoDB en carpeta de Google Drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os
db_path="/content/drive/MyDrive/BIG_DATA/DataBase/MongoDB/"
os.makedirs(db_path,exist_ok=True)
!sudo chmod 777 $db_path     #Permisos especiales sobre la carpeta

In [6]:
# Modificar variables de entorno del SO para que reconozca mongoDB
!sudo systemctl stop mongod
!sudo mongod --dbpath $db_path --fork --logpath /var/log/mongod.log

System has not been booted with systemd as init system (PID 1). Can't operate.
Failed to connect to bus: Host is down
about to fork child process, waiting until server is ready for connections.
forked process: 7491
child process started successfully, parent exiting


#3.Inicializar nuestro servidor de MongoDB

In [92]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)

##3.1.Crear una base de datos

In [82]:
# Crear la base de datos
db = client['estudiantes_ucentral']

In [83]:
# Crear colecciones
coleccion_profes = db['profesores']
coleccion_cursos = db['cursos']

##3.2.Funciones DML (insertar, borrar, actualizar)

In [72]:
def buscar_documentos(db, coleccion_nombre, filtro={}, limite=10):
  if db is not None:
      coleccion = db[coleccion_nombre]
      resultados = coleccion.find(filtro).limit(limite)
      for doc in resultados:
          print(doc)
  else:
      print("No hay conexión a la base de datos.")

def insertar_documento(db, coleccion_nombre, documento):
      if db is not None:
          coleccion = db[coleccion_nombre]
          resultado = coleccion.insert_one(documento)
          print(f"Documento insertado con ID: {resultado.inserted_id}")
      else:
          print("No hay conexión a la base de datos.")

def actualizar_un_documento(db, coleccion_nombre, filtro, actualizacion):
        if db is not None:
            coleccion = db[coleccion_nombre]
            resultado = coleccion.update_one(filtro, {'$set': actualizacion})
            if resultado.modified_count > 0:
                print(f"Documentos modificados: {resultado.modified_count}")
            else:
                print("No se encontraron documentos para modificar.")
        else:
            print("No hay conexión a la base de datos.")
def actualizar_varios_documentos(db, coleccion_nombre, filtro, actualizacion):
        if db is not None:
            coleccion = db[coleccion_nombre]
            resultado = coleccion.update_many(filtro, {'$set': actualizacion})
            if resultado.modified_count > 0:
                print(f"Documentos modificados: {resultado.modified_count}")
            else:
                print("No se encontraron documentos para modificar.")
        else:
            print("No hay conexión a la base de datos.")
def eliminar_varios_documentos(db, coleccion_nombre, filtro):
        if db is not None:
            coleccion = db[coleccion_nombre]
            resultado = coleccion.delete_many(filtro)
            if resultado.deleted_count > 0:
                print(f"Documentos eliminados: {resultado.deleted_count}")
            else:
                print("No se encontraron documentos para eliminar.")
        else:
            print("No hay conexión a la base de datos.")

In [73]:
# Insertar algunos documentos
documentoProfe1={"nombre":"luisfdo","apellidos":"castellanos guarin"}
documentoProfe2={"nombre":"pepito","apellidos":"perez","fechaNacimiento":"1980-05-05"}
insertar_documento(db,'profesores',documentoProfe1)
insertar_documento(db,'profesores',documentoProfe2)

Documento insertado con ID: 68c83aebe9fec699148ac3dc
Documento insertado con ID: 68c83aebe9fec699148ac3dd


In [84]:
buscar_documentos(db,'profesores')

{'_id': ObjectId('68c83aebe9fec699148ac3dc'), 'nombre': 'luisfdo', 'apellidos': 'castellanos guarin'}


In [77]:
actualizar_varios_documentos(db,'profesores',{"nombre":"pepito"},{"nombre":"Pedro Patricio"})

No se encontraron documentos para modificar.


In [78]:
buscar_documentos(db,'profesores')

{'_id': ObjectId('68c83aebe9fec699148ac3dc'), 'nombre': 'luisfdo', 'apellidos': 'castellanos guarin'}
{'_id': ObjectId('68c83aebe9fec699148ac3dd'), 'nombre': 'Pedro Patricio', 'apellidos': 'perez', 'fechaNacimiento': '1980-05-05'}


In [79]:
eliminar_varios_documentos(db,'profesores',{"apellidos":"perez"})
buscar_documentos(db,'profesores')

Documentos eliminados: 1
{'_id': ObjectId('68c83aebe9fec699148ac3dc'), 'nombre': 'luisfdo', 'apellidos': 'castellanos guarin'}


In [90]:
client.close()

#4.Descargar y descomprimir zip

In [85]:
def descargar_y_descomprimir_zip(url, carpeta_destino, tipoArchivo=''):
  import os
  import io
  import zipfile
  import requests
  os.makedirs(carpeta_destino, exist_ok=True)  #cree la carpeta sino existe
  response = requests.get(url)
  zip_file = zipfile.ZipFile(io.BytesIO(response.content))
  if (tipoArchivo == ''):
    zip_file.extractall(carpeta_destino) #exportar .zip a la carpeta
  else:
    for nombre_archivo in zip_file.namelist():
      if nombre_archivo.endswith(tipoArchivo):
        zip_file.extract(nombre_archivo, carpeta_destino)

In [86]:
carpeta_destino ="/content/drive/MyDrive/BIG_DATA/DataBase/zip_files/"
url             ="https://archive.ics.uci.edu/ml/machine-learning-databases/00320/student.zip"
descargar_y_descomprimir_zip(url,carpeta_destino)

#5.Procesar data descargada y cargar una a colección

In [87]:
import re
def procesar_txt_diccionario_separados_por_espacio(data_diccionario):
  lineas=data_diccionario.strip().split('\n')
  #---listar
  documentos=[]
  for linea in lineas:
    match = re.match(r'^\d+\s+(.+?)\s*-\s*(.+)$',linea)
    if match:
      clave= match.group(1)
      valor= match.group(2)
      documentos.append({"clave":clave, "valor":valor})
  return documentos

In [88]:
with open(carpeta_destino+'student.txt', 'r') as archivo:
    data_diccionario = archivo.read()
documentos=procesar_txt_diccionario_separados_por_espacio(data_diccionario)
print(documentos)

[{'clave': 'school', 'valor': 'student\'s school (binary: "GP" - Gabriel Pereira or "MS" - Mousinho da Silveira)'}, {'clave': 'sex', 'valor': 'student\'s sex (binary: "F" - female or "M" - male)'}, {'clave': 'age', 'valor': "student's age (numeric: from 15 to 22)"}, {'clave': 'address', 'valor': 'student\'s home address type (binary: "U" - urban or "R" - rural)'}, {'clave': 'famsize', 'valor': 'family size (binary: "LE3" - less or equal to 3 or "GT3" - greater than 3)'}, {'clave': 'Pstatus', 'valor': 'parent\'s cohabitation status (binary: "T" - living together or "A" - apart)'}, {'clave': 'Medu', 'valor': "mother's education (numeric: 0 - none,  1 - primary education (4th grade), 2 – 5th to 9th grade, 3 – secondary education or 4 – higher education)"}, {'clave': 'Fedu', 'valor': "father's education (numeric: 0 - none,  1 - primary education (4th grade), 2 – 5th to 9th grade, 3 – secondary education or 4 – higher education)"}, {'clave': 'Mjob', 'valor': 'mother\'s job (nominal: "teache

In [94]:
db = client['estudiantes_ucentral']
coleccion_estudiantes = db['estudiantes']
for documento in documentos:
  insertar_documento(db,'estudiantes',documento)

Documento insertado con ID: 68c83dede9fec699148ac3df
Documento insertado con ID: 68c83f76e9fec699148ac3e1
Documento insertado con ID: 68c83f76e9fec699148ac3e2
Documento insertado con ID: 68c83f76e9fec699148ac3e3
Documento insertado con ID: 68c83f76e9fec699148ac3e4
Documento insertado con ID: 68c83f76e9fec699148ac3e5
Documento insertado con ID: 68c83f76e9fec699148ac3e6
Documento insertado con ID: 68c83f76e9fec699148ac3e7
Documento insertado con ID: 68c83f76e9fec699148ac3e8
Documento insertado con ID: 68c83f76e9fec699148ac3e9
Documento insertado con ID: 68c83f76e9fec699148ac3ea
Documento insertado con ID: 68c83f76e9fec699148ac3eb
Documento insertado con ID: 68c83f76e9fec699148ac3ec
Documento insertado con ID: 68c83f76e9fec699148ac3ed
Documento insertado con ID: 68c83f76e9fec699148ac3ee
Documento insertado con ID: 68c83f76e9fec699148ac3ef
Documento insertado con ID: 68c83f76e9fec699148ac3f0
Documento insertado con ID: 68c83f76e9fec699148ac3f1
Documento insertado con ID: 68c83f76e9fec69914

In [95]:
buscar_documentos(db,'estudiantes')

{'_id': ObjectId('68c83dede9fec699148ac3df'), 'clave': 'school', 'valor': 'student\'s school (binary: "GP" - Gabriel Pereira or "MS" - Mousinho da Silveira)'}
{'_id': ObjectId('68c83f76e9fec699148ac3e1'), 'clave': 'sex', 'valor': 'student\'s sex (binary: "F" - female or "M" - male)'}
{'_id': ObjectId('68c83f76e9fec699148ac3e2'), 'clave': 'age', 'valor': "student's age (numeric: from 15 to 22)"}
{'_id': ObjectId('68c83f76e9fec699148ac3e3'), 'clave': 'address', 'valor': 'student\'s home address type (binary: "U" - urban or "R" - rural)'}
{'_id': ObjectId('68c83f76e9fec699148ac3e4'), 'clave': 'famsize', 'valor': 'family size (binary: "LE3" - less or equal to 3 or "GT3" - greater than 3)'}
{'_id': ObjectId('68c83f76e9fec699148ac3e5'), 'clave': 'Pstatus', 'valor': 'parent\'s cohabitation status (binary: "T" - living together or "A" - apart)'}
{'_id': ObjectId('68c83f76e9fec699148ac3e6'), 'clave': 'Medu', 'valor': "mother's education (numeric: 0 - none,  1 - primary education (4th grade), 2 

In [68]:
client.close()